# Lab: Bring your own script with Amazon SageMaker

## Sklearn script mode training and serving
Script mode is a training script format for a number of supported frameworks that lets you execute the training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native SKlearn support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). In this example, we will show how easily you can train a SageMaker using scikit-learn and with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker SKlearn container](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-docker-containers-scikit-learn-spark.html). 

## Set up the environment
Let's start by setting up the environment:

In [1]:
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import os
import tarfile
import pandas as pd

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
#prefix = 'sagemaker/DEMO-BYO'

role = sagemaker.get_execution_role()

## Training data
we download the Iris data from UCI Machine Learning repository directly from the web.

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

--2021-08-09 07:29:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data.1’

iris.data.1         100%[===================>]   4.44K  --.-KB/s    in 0s      

2021-08-09 07:29:35 (148 MB/s) - ‘iris.data.1’ saved [4551/4551]



## Train and test split
We split the data into train and test set

In [4]:
data = pd.read_csv('iris.data', 
                   names=['sepal length', 'sepal width', 
                          'petal length', 'petal width', 
                          'label'])

# Shuffle the data to make deploy_test samples random
data = data.sample(frac=1).reset_index(drop=True)
train = data[:-10]
test = data[-10:]
train.to_csv('train.csv')
test.to_csv('test.csv')
len(test)

10

Load our data to our S3 ready for training using our script.

In [5]:
import boto3

s3_session = boto3.Session().resource('s3')

s3_session.Bucket(bucket).Object('train/train.csv').upload_file('train.csv')
s3_session.Bucket(bucket).Object('test/test.csv').upload_file('test.csv')

## Construct a script for brining your own SKlearn script to SageMaker
Your Scikit-learn training script must be a Python 3.6 compatible source file.
The training script is similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables. For example:
- SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
- SM_OUTPUT_DATA_DIR: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.
- Supposing two input channels, ‘train’ and ‘test’, were used in the call to the Scikit-learn estimator’s fit() method, the following will be set, following the format “SM_CHANNEL_[channel_name]”:
- SM_CHANNEL_TRAIN: A string representing the path to the directory containing data in the ‘train’ channel
- SM_CHANNEL_TEST: Same as above, but for the ‘test’ channel.

In order to save your trained Scikit-learn model for deployment on SageMaker, your training script should save your model to a certain filesystem path called model_dir. This value is accessible through the environment variable SM_MODEL_DIR.

Load the model: before a model can be served, it must be loaded. The SageMaker Scikit-learn model server loads your model by invoking a 'model_fn' function that you must provide in your script

Serve a Model: after the SageMaker model server has loaded your model by calling model_fn, SageMaker will serve your model. Model serving is the process of responding to inference requests, received by SageMaker InvokeEndpoint API calls. The SageMaker Scikit-learn model server breaks request handling into three steps:

-input processing,
-prediction, and
-output processing.

Here is the entire script and you can see all these details explained above in the script.

In [6]:
!pygmentize 'BYO_sklearn_main.py'


"""
File: BYO_scikitlearn_model
"""

import argparse
import numpy as np
import os
import pandas as pd
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression


# Dictionary to encode labels to codes
label_encode = {
    'Iris-virginica': 0,
    'Iris-versicolor': 1,
    'Iris-setosa': 2
}

# Dictionary to convert codes to labels
label_decode = {
    0: 'Iris-virginica',
    1: 'Iris-versicolor',
    2: 'Iris-setosa'
}


if __name__ =='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--output-data-dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    #parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))

    args = parser.parse_args()

    data = pd.read_csv(os.path.join(args.train,'train.csv'),index_col=0, engin

In [7]:
sklearn_estimator = SKLearn('BYO_sklearn_main.py',
                            instance_type='ml.m4.xlarge',
                            framework_version='0.20.0',
                            role=role)


# Calling `fit`
To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. fit creates a default channel named 'training', which points to this S3 location. In the training script we can then access the training data from the location stored in SM_CHANNEL_TRAINING. fit accepts a couple other types of input as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

When training starts, the Scikit-learn container executes BYO_sklearn_main.py, passing hyperparameters and model_dir from the estimator as script arguments. Because we didn't define either in this example, no hyperparameters are passed, and model_dir defaults to `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, so the script execution is as follows:

`BYO_sklearn_main.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`

When training is complete, the training job will upload the saved model to S3 for deployment.

In [8]:
sklearn_estimator.fit({'train': 's3://{}/train/train.csv'.format(bucket),
                        'test': 's3://{}/test/test.csv'.format(bucket)})


2021-08-09 07:29:50 Starting - Starting the training job...
2021-08-09 07:30:14 Starting - Launching requested ML instancesProfilerReport-1628494190: InProgress
...
2021-08-09 07:30:46 Starting - Preparing the instances for training.........
2021-08-09 07:32:15 Downloading - Downloading input data...
2021-08-09 07:32:51 Training - Downloading the training image...
2021-08-09 07:33:18 Training - Training image download completed. Training in progress..2021-08-09 07:33:19,084 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-08-09 07:33:19,087 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-09 07:33:19,098 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-08-09 07:33:19,483 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-09 07:33:22,536 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-09 07

## Deploy
We are now ready to deploy our model to Sagemaker hosting services and make real time predictions

In [9]:
predictor = sklearn_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)


-------------!

Let's now send some data to our model to predict- the data shouldbe sent in the accepted format and the code below just does that.

In [10]:
test=pd.read_csv("test.csv").values.tolist()
test

[[140, 6.3, 2.3, 4.4, 1.3, 'Iris-versicolor'],
 [141, 6.3, 2.9, 5.6, 1.8, 'Iris-virginica'],
 [142, 5.4, 3.4, 1.5, 0.4, 'Iris-setosa'],
 [143, 6.8, 2.8, 4.8, 1.4, 'Iris-versicolor'],
 [144, 5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
 [145, 5.1, 3.7, 1.5, 0.4, 'Iris-setosa'],
 [146, 6.0, 3.4, 4.5, 1.6, 'Iris-versicolor'],
 [147, 6.2, 2.9, 4.3, 1.3, 'Iris-versicolor'],
 [148, 7.3, 2.9, 6.3, 1.8, 'Iris-virginica'],
 [149, 6.0, 3.0, 4.8, 1.8, 'Iris-virginica']]

In [11]:
request_body = ""
for row in test:
    request_body += ",".join([str(n) for n in row[1:-1]]) + "\n"
request_body = request_body[:-1]

print(request_body)

6.3,2.3,4.4,1.3
6.3,2.9,5.6,1.8
5.4,3.4,1.5,0.4
6.8,2.8,4.8,1.4
5.1,3.8,1.5,0.3
5.1,3.7,1.5,0.4
6.0,3.4,4.5,1.6
6.2,2.9,4.3,1.3
7.3,2.9,6.3,1.8
6.0,3.0,4.8,1.8


In [12]:
client = boto3.client('sagemaker-runtime')

endpoint=predictor.endpoint_name


content_type = "text/csv"

response = client.invoke_endpoint(
    EndpointName=endpoint,
    ContentType=content_type,
    Body=request_body
    )


In [13]:
response['Body'].read()

b'Iris-versicolor | Iris-virginica | Iris-setosa | Iris-versicolor | Iris-setosa | Iris-setosa | Iris-virginica | Iris-versicolor | Iris-virginica | Iris-virginica'

# Bring your own model saved on s3

In the previous section, we have import the model from training to hosting. In this section, we will demonstrate how to bring your own model to an endpoint without the training step. We will use the [SKLearnModel](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/sklearn/model.py#L67) class provided by SageMaker python sdk to create the model object. 

In [14]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = 'Iris-sklearn-byom-'+ strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_data = 's3://{}/{}/output/model.tar.gz'.format(bucket,sklearn_estimator.latest_training_job.job_name)

sklearn_model = SKLearnModel(
                    model_data=model_data,
                    role=role,
                    entry_point='BYO_sklearn_main.py',
                    framework_version='0.20.0')

In [15]:
%%time
predictor_byom = sklearn_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-----------------!CPU times: user 366 ms, sys: 27.4 ms, total: 394 ms
Wall time: 8min 32s


In [16]:
client = boto3.client('sagemaker-runtime')

endpoint_byom=predictor_byom.endpoint_name
print(endpoint_byom)

content_type = "text/csv"

response_byom = client.invoke_endpoint(
    EndpointName=endpoint,
    ContentType=content_type,
    Body=request_body
    )


sagemaker-scikit-learn-2021-08-09-07-40-36-837


In [17]:
response_byom['Body'].read()

b'Iris-versicolor | Iris-virginica | Iris-setosa | Iris-versicolor | Iris-setosa | Iris-setosa | Iris-virginica | Iris-versicolor | Iris-virginica | Iris-virginica'